## Introduction to Natural Language Processing
[**CC-BY-NC-SA**](https://creativecommons.org/licenses/by-nc-sa/4.0/deed.en)<br/>
Prof. Dr. Annemarie Friedrich<br/>
Faculty of Applied Computer Science, University of Augsburg<br/>
Date: **SS 2025**

# 6. Gradient Descent

**Learning Goals:**
1. Compute and implement tf.idf​
2. Derive binary cross entropy loss and its gradient​
3. Compute and implement cross entropy loss​
4. Explain and implement gradient descent



In [ ]:
# Imports and helper functions
import math
import numpy as np
from numpy import arange # for ranges with decimal steps
from math import log
import matplotlib
from matplotlib import pyplot as plt # for plotting

In our last session, we have learned about logistic regression classifiers. The configuration of the weights $w$ and $b$ was given and we have learned how to classify new instances by making use of the decision boundary that these weights define in the features space (or, equivalently, passing the scores through the sigmoid function and deciding based on the probability value).

However, in practice, if we have large and complicated datasets, we cannot simply guess a good decision boundary. Luckily, there are methods to compute it. Today, we will learn about one such method: __gradient descent__.

Let's assume we have the following training data. Each instance is represented using two feature values and one-hot encoded binary label.

In [ ]:
train_data = [([2, 1], [1,0]), # SPAM
              ([5,2], [1, 0]), # SPAM
              ([3, 4], [0, 1])] # NO_SPAM
train_data_x = [l[0] for l in train_data]
train_data_y = [l[1] for l in train_data]
print("training data x:", train_data_x)
print("training data y:", train_data_y)

In [ ]:
# Some helper functions and indices for plotting and inspecting results

# Given a one-hot encoded vector, return the class label as a string
# 0 means the first entry of the one-hot-encoded class label is 1 ==> [1, 0]
# 1 means the second entry of the one-hot-encoded class label is 1 ==> [0, 1]
idx2label = {1: "NO_SPAM", 0: "SPAM"}
idx2color = {1: "blue", 0: "red"} # we need this for plotting only

def get_label_string(one_hot_label):
  return idx2label[one_hot_label.index(1)]

def get_color_string(one_hot_label):
  return idx2color[one_hot_label.index(1)]


In addition, we know that we want to use a logistic regression classifier. This is a design decision that we need to make as a machine learning / NLP engineer or researcher. We could as well have decided to use a decision tree, but today we want to learn how to train logistic regression classifiers.

Logistic regression classifiers are defined by the weights $w = (w_1, w_2)$ where each weight deals with one feature dimension of instance $x$, and a bias $b$ (for binary logistic regression classifiers, this is just a scalar value). These are the three parameters that we need to train, i.e., we want to estimate good values for them.


In [ ]:
def classify(x, w, b):
  # Binary logistic regression classifier
  z = np.dot(x, w) + b # score
  print("in classify", z)
  prob = 1 / (1 + math.exp(-z)) # sigmoid function computes the probability of the positive class
  return prob


What should we choose as $w$ and $b$? As stated before, we could come up with a good guess for our toy dataset, but this is impossible if the dataset is huge and we have many features.

__Optimization Problem:__ We have defined our class of functions (here: a logistic regression classifier). We now want to find the $w$ and $b$ that results in a classifier that best fits the training data we happen to have at hand.

## Binary Cross Entropy

First, we need to define a __loss function__ that describes how well our classifier fits the training data. As mentioned in the lecture, this is different from computing the performance, we also aim to predict class label vectors that are as good as possible. When computing performance for classifiers, we only care about whether it makes the right decisions or not.

For this example, we will use __binary cross entropy__ (BCE), which is defined as the _negative log likelihood_ of the training instances. BCE's big sister, __cross entropy__ (CE) is defined as $CE = - \sum_i^C y_i \cdot log(p_i)$ where $y_i$ is the true class of instance $i$ and $p_i$ is the predicted probability for the true class.

Above, we have represented our labels y as one-hot-encoded vectors, in which only one dimension is 1. We can obtain this dimension from the vectors using: `class_idx = class_vector.index(1)`. E.g., for `[0,1]` this will return `1`. In the notation below (following Jurafsky & Martin, section 5.5), $y$ corresponds to whether the class index of the positive class (0 in our case) is 1.

$BCE = -[y^i \cdot log(\hat{y}^i) + (1-y^i) \cdot log(1-\hat{y}^i)]$

The term above is defined for one instance $i$; finally, we'll sum this over all training instances.
$y^i$ is the class index of the _true_ class of instance $i$, $\hat{y}^i$ is the predicted probability score for instance $i$ (in binary logistic expression, this expresses the probability of the instance belonging to the _positive_ class). Hence:
* If an instance's true class is the positive class ($y^i = 1$), its negative log likelihood is $-log(\hat{y}^i)$, the negative log of the predicted probability that it belongs to the positive class.
* If an instance's true class is the negative class ($y^i = 0$), its negative log likelihood is $-(log(1-\hat{y}^i)$, the probability that it belongs to the negative class (in the binary case, 1-probability that it belongs to the positive class).

❗Optional reading: [Understanding binary cross-entropy / log loss: a visual explanation](https://towardsdatascience.com/understanding-binary-cross-entropy-log-loss-a-visual-explanation-a3ac6025181a) by Daniel Godoy.

In [ ]:
def binary_cross_entropy_loss(train_data, w, b):
  """
  This function computes the BCE for the training data if our logistic regression
  model is parameterized by w and b.
  Note that this naive implementation will suffer from underflow in real-world
  settings (doesn't happen for our toy dataset). Toolkit implementations apply
  some tricks to avoid this.
  """
  ce = 0
  for x, label in train_data:
    prob = classify(x, w, b)
    ce -= label[0] * log(prob) + label[1] * log(1 - prob)
  return ce

# Let's try this
w = np.array([-0.2, 0.5])
b = -0.5
print("Error/Loss:", binary_cross_entropy_loss(train_data, w, b))

# What if we choose the parameters less wisely?
w = np.array([0.9, 0.5])
b = -0.5
print("Error/Loss:", binary_cross_entropy_loss(train_data, w, b))


Now we know that the error is quite small when using $w=[-0.2, 0.5]$ and $b=-0.5$ (there may be other values that make the loss go to 0, but in practice, we only care about finding one good weight parameterization). We also know that the error is smaller for $w=[-0.2, 0.5]$ and $b=-0.5$ compared to $w=[0.9,0.5]$ and $b=-0.5$. In theory, we could simply try out all potential weight parameters, but in practice, this is infeasible as we are usually trying to optimize not just one, but hundreds or thousands of parameters.

The training data is always fixed. __Important!__ As our training data is fixed, $x$ and the gold labels $y$ are _fixed_ parameters of our loss function, they will never change in training! The loss function is _not_ a function of $x$ or the gold labels $y$. What will change are just the weights $w$ and $b$. (The prediction $\hat{y}$ is computed using $w$ and $b$.) Hence, we can consider the loss function to be a function of the weights $w$.

As a simple example, let's keep $w_2$ and $b$ fixed.
We will set them both to 0.5, i.e., we only aim to find a really good value for $w_1$ in case $w_2$ and $b$ are "frozen" (for whatever reason). Let's plot the loss function depending on the value of $w_1$.

The input to the BCE is our entire training data and the fixed parameters. It returns the loss value for any $w_1$ that we check. (Note: in this case, you could compute the solution analytically. But even for our toy dataset, it is quite tedious.)

In [ ]:
def plot_loss_function():

  w2 = 0.5 # fixed
  b = 0.5 # fixed

  # Let's try lots of values for w1 and compute the error
  # (This is impossible in practice for larger models!)
  weights, loss_values = [], []
  for w1 in np.arange(-4, 4, 0.01):
    weights.append(w1) # This will be the x-values in the plot
    # Compute the corresponding y-values:
    loss_values.append(binary_cross_entropy_loss(train_data, [w1, w2], b))
    # Note that the input to the binary cross entropy loss function is the entire training data,
    # plus our fixed weights.

  # Let's plot this
  plt.rcParams["figure.figsize"] = [4, 4]
  font = {'weight' : 'bold',
        'size'   : 16}

  matplotlib.rc('font', **font)
  plt.xlim(-4, 4)
  plt.ylim(0, 20)
  plt.grid()
  plt.xlabel("$w_1$")
  plt.ylabel("loss")

  # loss function
  plt.plot(weights, loss_values)

plot_loss_function()
plt.show()

And that is how what our loss function looks like. Remember that it is a _negative_ log likelihood, i.e., we aim to _minimize_ this loss. If we choose a really good $w_1$, the predicted class probability vectors will be very similar to the gold standard class vectors. The ideal $w_1$ is a bit less than 0. But how do we find the exact value?


### Slopes, Derivatives, Gradients

Let's start at some random point, e.g., $w_1=2$. If you put a marble on the graph at that point, it will go downwards. It might oscillate a bit at the bottom, but finally it will stop at the lowest point of our function. This is what __gradient descent__ does, just mathematically instead of using gravity. As math isn't as perfect as gravity, we might sometimes end up not quite at the bottom, but usually very closely.

How can we model gravity using math? Imagine putting the marble at the point with $w_1=2$. In which direction would the marble go? It would go to the left, because the function is going upwards at that point, i.e., its _slope_ is positive. In our numeric setting, we are just given the training data and the BCE formula.


Luckily, the BCE formula is _differentiable_, which means that we can compute its first derivative (= the slope) at the point defined by $w_1 = 2$. Let's look at how this can be done.

$BCE = -[y^i \cdot log(\hat{y}^i) + (1-y^i) \cdot log(1-\hat{y}^i)]$

$ = -[y^i \cdot log(\sigma(w_1 \cdot x_1 + w_2 \cdot x_2 + b)) + (1-y^i) \cdot log(\sigma(1 - (w_1 \cdot x_1 + w_2 \cdot x_2 + b)))]$

The derivative of this negative log likelihood with respect to parameter $w_2$ is:

$\displaystyle \frac{d}{d w_2} = [\sigma(w \cdot x + b) - y^i] \cdot x_2$

Recall that $\sigma(w\cdot x + b)$ is the probability value (`prob` below) that instance $i$ belongs to the positive class. The scalar $y^i$ is the class index (0 or 1 for binary classification) of the true class of instance $i$.

To get the full loss value, we sum the value of this expression for each training instance.
For the complete derivation of this derivative, see Jurafsky & Martin, Speech and Language Processing, section 5.10 (optional reading).

The function for computing the slope below is called `gradient` as you will more commonly encounter this terminology. A gradient is a vector of partial derivates. Here, we have the special case of only one parameter and one derivative.

❓Take a look at the plot of the loss function again. Then, use the `print` statements below to compute the slope for a few different values of $w_1$. Do the values match the slopes in the plot? In the plot below, the dark blue line is the slope at for the w1-value indicated by the orange line.

In [ ]:
def gradient(w1):
  # Computes slope (derivative) of the loss function at w1.
  w2=0.5 # fixed
  b=0.5 # fixed
  gradient = 0
  for x, label in train_data:
    prob = classify(x, [w1, w2], b)
    gradient += (prob - label[0]) * x[0]
  return gradient

w1 = -1.6 # CHANGE THIS VALUE
print(w1, gradient(w1))

# Let's plot this
plot_loss_function()
plt.xlim(-2, 2)
plt.ylim(0, 10)
plt.plot([w1,w1], [0,10])
w2_fixed = 0.5
b_fixed = 0.5

# Gradient at our current w
print("Gradient at", w1, "is", gradient(w1), " and has loss value ", binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed))
# Plot this nicely
plt.plot([w1, w1+1], [binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed), binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed)+gradient(w1)], color="blue")
plt.plot([w1, w1-1], [binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed), binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed)-gradient(w1)], color="blue")

plt.show()

# Hint: the dark blue line is the slope at for the w1-value indicated by the orange line.

### Finally: Gradient Descent

Now, we have all the ingredients we need to find the optimal (or at least a very good) value for $w_1$.

Gradient descent now simulates the rolling of the marble as follows.
* `loss_function` is the choice of our loss function, i.e., gradient descent needs to know how we want to compute the loss. We have seen binary cross entropy above, but there are modifications to cross entropy or completely different loss functions. They tell the optimization algorithm what should be achieved, as they are a distance function between the training data and the predictions.
* `classifier` is the classifier that predicts the scores or probabilites per instance. It is parameterized by the weights and biases that we aim to learn. We want to optimize the parameters of this classifier.
* `train_data` are the feature representations / values and the class labels of the training data.


```
gradient_descent(loss_function, classifier, train_data):

  Initialize the parameters Θ randomly (or with some good guess).
  Choose learning_rate.

  Repeat until loss converges (does not get better any more):

    # Optional, for reporting only:
    loss = 0
    For each training instance [x^i, y^i] -- in random order
      Compute prediction ŷ^i = classifier(x^i,Θ)
      loss = loss+ loss_function(^y^i, y^i) # sum loss of all instances

    # Must do:
    Compute the gradient g (using training data and current Θ).
    Update the weights: Θ = Θ - learning_rate * g

  return Θ

```

* Here, we view the set of weights and biases as a parameter vector `Θ`.
* The gradient `g` is the vector of all partial derivates.
* The `learning rate` is a hyperparameter, i.e., a parameter that we have to define manually before the learning takes place. It tells the optimizer how big the steps should be that we take in the update. Imagine that our marble does not roll down the hill smoothly but that it is actually a bit more like a rubber ball taking little hops. The learning rate is how far the ball jumps in each step.
* The slope (gradient) tells us in which direction the loss function moves _up_ at a given value of $w_1$. Recall that we do not want to move up, but that we aim to minimize the loss, i.e., we want to move _down_. How can we fix this mathematically? Easy, we just multiply this direction by -1. That is why we subtract the `learning_rate * g` in the weight update step above.

And that's the core idea of gradient descent, with the math for the binary logistic regression case.

❗Make sure to work through the steps carefully, as they are the basis for the neural networks that we will discuss. The logistic regression classifier can be viewed as the smallest possible neural network (with just one node).

After this much math, you deserved it: Let's take a look at gradient descent in __action!__

The code below executes gradient descent for our classifier and training dataset as described above. It assumes $w_2$ and $b$ to be frozen (such that we can actually plot the loss function and the slope)).

❓Execute the cell below. It will perform the entire gradient descent. Did it end up at the minimum of the loss function? What is its value? What is the slope at this point? What should it be (in theory)?

❓Change the parameter max_steps to follow some steps during learning. The algorithm will then step after performing this many update steps. I recommend that you try 1, 2, 4, 6, 10, ... and take a look at the plot, which will show the state of the algorithm at these learning steps. The orange line depicts the current value of $w_2$.

In [ ]:
# Let's put everything together (stopping if error increases again for now)
def gradient_descent(train_data, initial_w1, learning_rate, max_steps=None):
  w2_fixed = 0.5
  b_fixed = 0.5

  w1 = initial_w1
  prev_error = None
  curr_error = binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed)
  s = 0
  while prev_error is None or curr_error < prev_error:
    step = -1 * learning_rate * gradient(w1) # the minus one because positive slope means we need to step backwards!
    w1 = w1 + step
    print("Step", s, "Next recommended w = {:1.5f}".format(w1), "with error = ", curr_error)
    prev_error = curr_error
    curr_error = binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed)
    s += 1
    if max_steps is not None and s >= max_steps:
      return w1, curr_error
  return w1, curr_error

### MAKE YOUR CHANGES HERE!! ###############################
learning_rate = 0.1 # try 0.01, 0.2, 0.4, ...
max_steps = 27 # you can change this to 1, 2, 3, ... to see the individual steps taken during learning.
############################################################

w1 = 3 # initialization
w1, curr_error = gradient_descent(train_data, w1, learning_rate, max_steps)
print("Final recommended w =", w1, "with error = ", curr_error)

plot_loss_function()
plt.xlim(-4, 4)
plt.ylim(0, 20)
plt.plot([w1,w1], [0,20])
w2_fixed = 0.5
b_fixed = 0.5

# Gradient at our current w
print("Gradient at", w1, "is", gradient(w1), " and has loss value ", binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed))
# Plot this nicely
plt.plot([w1, w1+1], [binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed), binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed)+gradient(w1)], color="blue")
plt.plot([w1, w1-1], [binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed), binary_cross_entropy_loss(train_data, [w1, w2_fixed], b_fixed)-gradient(w1)], color="blue")

plt.show()

❓The __learning rate__ defines the size of the steps taken during the weight updates. Try different values for the learning rate. What happens? In which cases does the optimizer find the correct minimum? How long does the training take? In which cases does it fail? Why is this the case? Note down the values and your findings (including the final loss) in a table. Which is the best learning rate according to your findings? (You just performed _hyperparameter tuning_.)


❓The pseudocode above describes _batch_ gradient, which performs updates only after computing the loss and gradient for the entire training dataset.
Section 5.6.2 in Jurafsky & Martin, Speech and Language Processing, describes the Stochastic Gradient algorithm. Take a note here how they differ. Is there an algorithm combining the advantages of the two approaches?

### Learning with Many Weights/Parameters

❗Above, we have seen an example for gradient descent that updates a single parameter. In practice, we work with logistic regression classifiers that have many features and just as many weights. For those, we compute the _partial derivates_ of the loss function with respect to the weights. Intuitively, each parameter is a dimension of the loss function's space, and we the partial derivates tell us how to change the particular weight to decrease the loss function's value. We will leave the advanced math on this to the machine learning course, but we will see it in action when we talk about neural networks soon.